In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# some basic imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import scipy.stats as st
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Jordan/Data"

all_data_takaful_and_commercial.dta  new_choice_drugs.csv
choice_drug_not_matched1.csv	     Not_matched_drugs2.csv
choice_drug_not_matched2.csv	     Not_matched_drugs3.csv
classes.dta			     numclaims_switchdown_stayhigh.csv
combined_data.dta		     numclaims_switchdown_staylow.csv
combined_data_RIDITScores.csv	     numclaims_switchup_stayhigh.csv
datetest.csv			     numclaims_switchup_staylow.csv
drug_data.dta			     outcome.csv
Drug_search.xlsx		     percentile_scores2.csv
FinalCodedData.csv		     Plan_Switching_NetworkCoded.csv
final_data.csv			     s_stats.csv
FinalSwitchCodedData.csv	     Stay_high.csv
FinalSwitchCodedData_v1.csv	     Switchig_dataset.csv
Matched_drugs2.gsheet		     Switchig_dataset.dta
Matched_drugs4.csv		     Switching_dataset.csv
Matched_drugs5.csv		     transformed_data5.csv
merged_all1415.dta		     unique_drug_freq.csv


In [4]:

#changing the directory to the data directory
%cd /content/drive/My\ Drive/Jordan/Data

/content/drive/My Drive/Jordan/Data


In [9]:
class DataShell:
  def __init__(self):
    """Initializes the class with some information about the data and the model"""
    self.data_used = 'The data used is the Switch coded data to generate summary statistics'

  def read_data_stata(self,filename):
    self.data = pd.read_stata(filename)
    return self.data
  
  def read_data_csv(self,filename):
    self.data = pd.read_csv(filename)
    return self.data
  
  def initialize_subsets(self):
    self.switch_up = self.data[self.data['switch_up']==1]
    self.switch_down = self.data[self.data['switch_down']==1]
    self.stay_high = self.data[self.data['stay_high']==1]
    self.stay_low = self.data[self.data['stay_low']==1]
    self.add = self.data[self.data['addition']==1]

  def convert_to_datetime(self,column):
    """Converts a column to a datetime object
       parameters:
              column: The column to be converted to a datetime object
       returns: returns the dataframe with the converted column"""
    self.data[column] = pd.to_datetime(self.data[column],errors='coerce')
    return self.data

  def prepost_addition(self):
    self.data['prepost_addition'] = 0
    self.data1 = self.data
    self.data1.sort_values('principal_hof',inplace=True)
    self.add = self.data1[self.data1['addition']==1]
    self.add = self.add.reset_index(drop=True)
    self.noadd = self.data1[~self.data1['addition']==0]
    self.noadd = self.noadd.reset_index(drop=True)
    i = 0
    k1 = pd.DataFrame(columns=self.data1.columns)
    uni_prin_hof = self.add['principal_hof'].unique()
    val_counts = self.add['principal_hof'].value_counts().to_dict() 
    for j,p_hof in enumerate(uni_prin_hof):
      print(j/len(uni_prin_hof)*100)
      k2 = self.add[i:i+val_counts[p_hof]]
      k2.sort_values('endo_date',inplace=True)
      k2 = k2.reset_index(drop=True)
      for t in range(len(k2)):
        if k2.loc[t,'addition'] == 1:
          k2.loc[t:len(k2)-1,'prepost_addition'] = 1
          break
      k1 = k1.append(k2)
      i = i + val_counts[p_hof]
    self.data2 = self.noadd.append(k1,ignore_index=True)
    return self.data2

  def generate_summary(self,type1,type2,var):
    if type1 == 'switch_up':
      self.k = pd.DataFrame(self.switch_up[self.switch_up['prepostswitch_up']==0][var].describe())
      self.k1 = pd.DataFrame(self.switch_up[self.switch_up['prepostswitch_up']==1][var].describe())
    if type1 == 'switch_down':
      self.k = pd.DataFrame(self.switch_down[self.switch_down['prepostswitch_down']==0][var].describe())
      self.k1 = pd.DataFrame(self.switch_down[self.switch_down['prepostswitch_down']==1][var].describe())     
    if type1 == 'addition':
      self.k = pd.DataFrame(self.add[self.add['prepost_addition']==0][var].describe())
      self.k1 = pd.DataFrame(self.add[self.add['prepost_addition']==1][var].describe())     
    if type2 == 'stay_high':
      self.k2 = pd.DataFrame(self.stay_high[var].describe())
    if type2 == 'stay_low':
      self.k2 = pd.DataFrame(self.stay_low[var].describe())
    self.new = self.k.merge(self.k2,left_index=True,right_index=True)
    self.new.columns = ['Pre'+type1,type2]
    self.new.reset_index(inplace=True)
    self.new1 = self.k1.merge(self.k2,left_index=True,right_index=True)
    self.new1.columns = ['Post'+type1,type2]
    self.new1.reset_index(inplace=True)

  def gen_uphigh(self,type1,type2,var):
    if type1 == 'switch_up':
      sw = 'prepostswitch_up'
      df1 = self.switch_up
    if type1 == 'switch_down':
      sw = 'prepostswitch_down'
      df1 = self.switch_down
    if type1 == 'addition':
      sw = 'prepost_addition'
      df1 = self.add
    if type2 == 'stay_high':
      df2 = self.stay_high
    if type2 == 'stay_low':
      df2 = self.stay_low
    if st.levene(np.array(df1[df1[sw]==1][var]), np.array(df2[var]))[1] >0.05:
      st.ttest_ind(np.array(df1[df1[sw]==1][var]),np.array(self.df2[var]))[1]
      self.new1['p-value'] = st.ttest_ind(np.array(df1[df1[sw]==1][var]),np.array(df2[var]))[1]
      self.new1.loc[1:len(self.new1),'p-value'] = np.nan
    else:
      st.ttest_ind(np.array(df1[df1[sw]==0][var]),np.array(df2[var]),equal_var=False)[1]
      self.new1['p-value'] = st.ttest_ind(np.array(df1[df1[sw]==1][var]),np.array(df2[var]))[1]
      self.new1.loc[1:len(self.new1),'p-value'] = np.nan
    if st.levene(np.array(df1[df1[sw]==0][var]), np.array(df2[var]))[1] >0.05:
      st.ttest_ind(np.array(df1[df1[sw]==0][var]),np.array(df2[var]))[1]
      self.new['p-value'] = st.ttest_ind(np.array(df1[df1[sw]==1][var]),np.array(df2[var]))[1]
      self.new.loc[1:len(self.new),'p-value'] = np.nan
    else:
      st.ttest_ind(np.array(df1[df1[sw]==0][var]),np.array(df2[var]),equal_var=False)[1]
      self.new['p-value'] = st.ttest_ind(np.array(df1[df1[sw]==1][var]),np.array(df2[var]))[1]
      self.new.loc[1:len(self.new),'p-value'] = np.nan
    self.summary = self.new.merge(self.new1,left_index=True,right_index=True)
    return self.summary




In [24]:
df1=data_shell.add
sw = 'prepost_addition'
df2=data_shell.stay_high
var = 'avgclaimamt'
st.levene(np.array(df1[df1[sw]==1][var]), np.array(df2[var]))

LeveneResult(statistic=49.350453217066395, pvalue=2.1448730093059673e-12)

In [25]:
st.ttest_ind(np.array(df1[df1[sw]==0][var]),np.array(df2[var]),equal_var=False)

Ttest_indResult(statistic=37.69461040664116, pvalue=1.9098683241587212e-307)

In [10]:
#Creates an object for the class and reads the data from the directory
data_shell = DataShell()
data = data_shell.read_data_csv('Switching_dataset.csv')
data.head()

,Unnamed: 0,Unnamed: 0.1,avgclaimamt,numclaims,prepostswitch_down,prepostswitch_up,switch_type,switch_down,switch_up,addition,stay_high,stay_low,male,married,dependent,chronic,outpatient,education,policy_length,coverageamnt,lncoverage,lognumclaims,logclaim,claims_polmonth,claimamt_month,avgcopay,avgcoinsurance,avgdeductionamt,icd9chapters,vip_plan,economy_plan,icdchapter1,icdchapter2,icd_general,icdchapter22,nslash,pol_expiration,effictivedate,expirydate,dischargedate_month,...,providertype,providergroup,provider,claimedamount,beneficiartshe_coinsurance,totalcimaiedam,totalpayable,payershare,ben_share_approved,deductionamount,recovery_gartia,recoverytype,payable_non_ip_con,po,orderdate,cheque_nbr,settledate,status,claimcatogory,icdchapter,icd_subchapter,assessment,recoverynote,month,pol_effective,endo_day,exp_date_month,expirydate2,exp_date_year,effective_month,effictivedate22,effective_year,effictivedate2,twoyears,twoyrkeep,claimbin,college,college_degree,coverage,prepost_addition
0,0,538112,300.87,9.0,0,0,2.0,0,0,0,0,1,1.0,1.0,0.0,0.0,1.0,1.0,20.0,250000.0,12.429216,2.197225,5.706678,0.45,15.0435,25.193333,25.193333,47.666668,8.0,0,1,Diseases of the nervous system and Sense organs,"ICD9CM 320-389,ICD10CM G00-H95)",ICD9CM 320-389,ICD10CM G00-H95),0.0,NaN,6/1/2015,6/1/2016,11.0,...,Pharmacy,Aster Group,Aster Pharmacy 5 (Br of Aster Pharmacies Group...,11.66,1.3,12.96,11.66,11.66,1.3,0.0,0.0,,0.0,108157.0,NaN,NaN,NaN,PO Issued,To Be Paid,Diseases of the nervous system and Sense organ...,Disorders of conjunctiva (H10-H11) (372-372),"H11.32 Conjunctival hemorrhage, left eye",NaN,20,NaN,NaN,6.0,1.0,2016.0,6.0,1.0,2015.0,6/1/2015,NaN,NaN,1.0,yes,yes,1.0,0
1,1,538114,300.87,9.0,0,0,2.0,0,0,0,0,1,1.0,1.0,0.0,0.0,1.0,1.0,20.0,250000.0,12.429216,2.197225,5.706678,0.45,15.0435,25.193333,25.193333,47.666668,1.0,0,1,Diseases of the respiratory system,"ICD9CM 460-519,ICD10CM J00-J99)",ICD9CM 460-519,ICD10CM J00-J99),0.0,NaN,6/1/2015,6/1/2016,9.0,...,Polyclinic/Diagnostic Center,Aster Group,Al Rafa Polyclinic International City - Dubai,47.50,12.0,59.50,47.50,47.50,12.0,0.0,0.0,,1.0,106414.0,NaN,195336,1/28/2016,Settled,Paid,Diseases of the respiratory system (ICD9CM 460...,Chronic lower respiratory diseases (J40-J47)(4...,"J40 Bronchitis, not specified as acute or chronic",NaN,20,NaN,NaN,6.0,1.0,2016.0,6.0,1.0,2015.0,6/1/2015,NaN,NaN,1.0,yes,yes,1.0,0
2,2,538113,300.87,9.0,0,0,2.0,0,0,0,0,1,1.0,1.0,0.0,0.0,1.0,1.0,20.0,250000.0,12.429216,2.197225,5.706678,0.45,15.0435,25.193333,25.193333,47.666668,8.0,0,1,Diseases of the nervous system and Sense organs,"ICD9CM 320-389,ICD10CM G00-H95)",ICD9CM 320-389,ICD10CM G00-H95),0.0,NaN,6/1/2015,6/1/2016,11.0,...,Hospital,Aster Group,Dr. Moopen Polyclinic Br of DM Healthcare LLC,183.00,30.0,213.00,150.00,150.00,30.0,33.0,0.0,,1.0,110382.0,NaN,NaN,NaN,PO Issued,To Be Paid,Diseases of the nervous system and Sense organ...,Disorders of conjunctiva (H10-H11) (372-372),"H11.32 Conjunctival hemorrhage, left eye",NaN,20,NaN,NaN,6.0,1.0,2016.0,6.0,1.0,2015.0,6/1/2015,NaN,NaN,1.0,yes,yes,1.0,0
3,3,538115,300.87,9.0,0,0,2.0,0,0,0,0,1,1.0,1.0,0.0,0.0,1.0,1.0,20.0,250000.0,12.429216,2.197225,5.706678,0.45,15.0435,25.193333,25.193333,47.666668,4.0,0,1,"Symptoms, signs and abnormal clinical and labo...","ICD9CM 780-799,ICD10CM R00-R99)",ICD9CM 780-799,ICD10CM R00-R99),0.0,NaN,6/1/2015,6/1/2016,NaN,...,Polyclinic/Diagnostic Center,Aster Group,Medinova Diagnostic Center - Dubai,900.00,0.0,900.00,900.00,900.00,0.0,0.0,0.0,,0.0,NaN,NaN,,NaN,Authorized,Outstanding/Open Authorization,"Symptoms, signs and abnormal clinical and labo...",General symptoms and signs (R50-R69)(780.60-78...,R51 Headache,NaN,20,NaN,NaN,6.0,1.0,2016.0,6.0,1.0,2015.0,6/1/2015,NaN,NaN,1.0,yes,yes,1.0,0
4,4,538116,300.87,9.0,0,0,2.0,0,0,0,0,1,1.0,1.0,0.0,0.0,1.0,1.0,20.0,250000.0,12.429216,2.197225,5.706678,0.45,15.0435,25.193333,25.193333,47.666668,3.0,0,1,Diseases of the musculoskeletal system and con...,"ICD9CM 710-739,ICD10CM M00-M99)",ICD9CM 710

In [11]:
data = data_shell.convert_to_datetime('endo_date')
data = data_shell.convert_to_datetime('exp_date')
#data = data_shell.convert_to_datetime('effictive_date')
data = data_shell.convert_to_datetime('issue_date')

In [12]:
data_shell.initialize_subsets()
data_shell.generate_summary('addition','stay_high','avgclaimamt')
uphigh = data_shell.gen_uphigh('addition','stay_high','avgclaimamt')
uphigh.to_csv('avgclaimamt_addition_stayhigh.csv')

In [13]:
data_shell.generate_summary('addition','stay_low','avgclaimamt')
uphigh = data_shell.gen_uphigh('addition','stay_low','avgclaimamt')
uphigh.to_csv('avgclaimamt_addition_staylow.csv')

In [14]:
data_shell.generate_summary('addition','stay_high','numclaims')
uphigh = data_shell.gen_uphigh('addition','stay_high','numclaims')
uphigh.to_csv('numclaims_addition_stayhigh.csv')

In [15]:
data_shell.generate_summary('addition','stay_low','numclaims')
uphigh = data_shell.gen_uphigh('addition','stay_low','numclaims')
uphigh.to_csv('numclaims_addition_staylow.csv')

In [ ]:
data_shell.generate_summary('switch_up','stay_high','avgclaimamt')
uphigh = data_shell.gen_uphigh('switch_up','stay_high','avgclaimamt')
uphigh.to_csv('avgclaimamt_switchup_stayhigh.csv')

In [ ]:
data_shell.generate_summary('switch_up','stay_high','numclaims')
uphigh = data_shell.gen_uphigh('switch_up','stay_high','numclaims')
uphigh.to_csv('numclaims_switchup_stayhigh.csv')

In [ ]:
data_shell.generate_summary('switch_up','stay_low','numclaims')
uphigh = data_shell.gen_uphigh('switch_up','stay_low','numclaims')
uphigh.to_csv('numclaims_switchup_staylow.csv')

In [ ]:
data_shell.generate_summary('switch_up','stay_low','avgclaimamt')
uphigh = data_shell.gen_uphigh('switch_up','stay_low','avgclaimamt')
uphigh.to_csv('avgclaimamt_switchup_staylow.csv')

In [ ]:
data_shell.generate_summary('switch_down','stay_high','numclaims')
uphigh = data_shell.gen_uphigh('switch_down','stay_high','numclaims')
uphigh.to_csv('numclaims_switchdown_stayhigh.csv')

In [ ]:
data_shell.generate_summary('switch_down','stay_high','avgclaimamt')
uphigh = data_shell.gen_uphigh('switch_down','stay_high','avgclaimamt')
uphigh.to_csv('avgclaimamt_switchdown_stayhigh.csv')

In [ ]:
data_shell.generate_summary('switch_down','stay_low','numclaims')
uphigh = data_shell.gen_uphigh('switch_down','stay_low','numclaims')
uphigh.to_csv('numclaims_switchdown_staylow.csv')

In [ ]:
data_shell.generate_summary('switch_down','stay_low','avgclaimamt')
uphigh = data_shell.gen_uphigh('switch_down','stay_low','avgclaimamt')
uphigh.to_csv('avgclaimamt_switchdown_staylow.csv')

In [ ]:
k = pd.DataFrame(switch_up[switch_up['prepostswitch_up']==0].avgclaimamt.describe())
k1 = pd.DataFrame(switch_up[switch_up['prepostswitch_up']==1].avgclaimamt.describe())
new = k.merge(k1,left_index=True,right_index=True)
new.columns = ['PreSwitchUp','PostSwitchUp']
new.reset_index(inplace=True)
new

,index,PreSwitchUp,PostSwitchUp
0,count,4467.000000,8944.000000
1,mean,567.028320,601.227112
2,std,456.171539,1560.935181
3,min,0.000000,0.000000
4,25%,299.761658,281.128082
5,50%,466.410004,476.619629
6,75%,667.111023,733.005676
7,max,4809.381836,140618.015625
